<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0_csv%E4%BF%9D%E5%AD%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/株式投資')

Mounted at /content/drive


In [3]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=01d04eed40d555edf4e772882b085400e33afb5c5daab7049dbdcceac8bbf481
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
!pip install yahoo_finance_api2

  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=46243868ab6c12968795c56279925d647099e2e4a46796f031a98948ea2688dd
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [5]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

# 東証上場の銘柄一覧を取得する関数（ダウンロードはしない）

In [6]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    # サイトからダウンロードしないようにコメントアウト
    #url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    #r = requests.get(url)
    #with open('data_j.xls', 'wb') as output:
    #    output.write(r.content)

    df = pd.read_excel("./data/data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [7]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})

    close = df['Close']

    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)

    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False


    result = all(conditions)

    return result

In [8]:
def filtered_ticker(df, min_volume=None, max_price=None, min_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param min_price: 株価の最小値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    result_list = []

    # データフレームを辞書のリストに変更
    df_to_dicts = df.to_dict(orient='records')

    # 各銘柄についてループ
    for row in df_to_dicts:
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue

        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        if min_price is not None and price < min_price:
            continue

        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        result_list.append({
            '銘柄名': row['銘柄名'],
            'コード': code,
            '業種': row['業種'],
            '出来高': volume,
            '株価': price,
        })

    # 辞書のリストをデータフレームに変換
    result_df = pd.DataFrame.from_dict(result_list)

    return result_df

# CSVに保存する関数

In [55]:
import datetime as dt
import os
import pytz

jst = pytz.timezone('Asia/Tokyo')
now = dt.datetime.now(jst)
now = now.strftime("%Y%m%d_%H%M%S")
# ディレクトリ名作成（YYYYMMDD_HHMMSS_screening）
directory = f'{now}_screening'

def save_df_as_csv(df, filename):
    """データフレームをCSVファイルとして保存する

    :param df: 対象のデータフレーム
    :param filename: 保存するcsvファイル名
    """
    # もしディレクトリが存在しなければ、作成する
    if not os.path.exists(directory):
        os.makedirs(directory)

    # データフレームをCSVに保存する
    df.to_csv(f'{directory}/{filename}.csv', index=False)

# 東証プライムをフィルタリングしてCSVとして保存

In [10]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


In [11]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
import time
start_time = time.time()
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=500000, max_price=5000)
print(f'経過時間: {time.time() - start_time}秒')
filtered_tse_prime

'timestamp'
銘柄コード[1954]の株価情報を取得できなかったため、スキップしました。
Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5128]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6924]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9810]の株価情報を取得できなかったため、スキップしました。
経過時間: 149.09652471542358秒


,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1126800,646.099976
1,ＩＮＰＥＸ,1605,鉱業,5526500,1598.000000
2,安藤・間,1719,建設業,562900,1075.000000
3,コムシスホールディングス,1721,建設業,531400,2844.500000
4,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,574000,31.000000
...,...,...,...,...,...
336,エイチ・アイ・エス,9603,サービス業,1553000,2038.000000
337,エヌ・ティ・ティ・データ,9613,情報・通信業,2963000,2007.000000
338,ＳＣＳＫ,9719,情報・通信業,588300,2259.500000
339,ヤマダホールディングス,9831,小売業,3978800,424.399994


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1126800,646.099976
1,ＩＮＰＥＸ,1605,鉱業,5526500,1598.000000
2,安藤・間,1719,建設業,562900,1075.000000
3,コムシスホールディングス,1721,建設業,531400,2844.500000
4,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,574000,31.000000
...,...,...,...,...,...
175,東北電力,9506,電気・ガス業,2043500,890.200012
176,四国電力,9507,電気・ガス業,530800,981.400024
177,九州電力,9508,電気・ガス業,1208000,920.500000
178,北海道電力,9509,電気・ガス業,930500,592.000000


In [13]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1126800,646.099976
1,ＩＮＰＥＸ,1605,鉱業,5526500,1598.000000
2,安藤・間,1719,建設業,562900,1075.000000
3,コムシスホールディングス,1721,建設業,531400,2844.500000
4,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,574000,31.000000
...,...,...,...,...,...
128,中部電力,9502,電気・ガス業,2268800,1757.000000
129,関西電力,9503,電気・ガス業,2441300,1805.500000
130,北陸電力,9505,電気・ガス業,672300,775.299988
131,東北電力,9506,電気・ガス業,2043500,890.200012


In [14]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_prime, '1_東証P_UPO')

In [15]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
0,三井住友建設,1821,建設業,572800,370.000000
1,パーソルホールディングス,2181,サービス業,1038000,2589.000000
2,明治ホールディングス,2269,食料品,833100,3218.000000
3,ベネフィット・ワン,2412,サービス業,1301600,1465.000000
4,エムスリー,2413,サービス業,2081000,3109.000000
5,エスプール,2471,サービス業,670400,527.000000
6,伊藤園,2593,食料品,651900,3969.000000
7,ユーグレナ,2931,食料品,588700,890.000000
8,ラクーンホールディングス,3031,情報・通信業,986700,681.000000
9,ビックカメラ,3048,小売業,514000,1068.000000


In [16]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime, perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
0,パーソルホールディングス,2181,サービス業,1038000,2589.000000
1,ベネフィット・ワン,2412,サービス業,1301600,1465.000000
2,エムスリー,2413,サービス業,2081000,3109.000000
3,エスプール,2471,サービス業,670400,527.000000
4,伊藤園,2593,食料品,651900,3969.000000
5,ラクーンホールディングス,3031,情報・通信業,986700,681.000000
6,ＫＬａｂ,3656,情報・通信業,1128900,311.000000
7,ｇｕｍｉ,3903,情報・通信業,981100,636.000000
8,協和キリン,4151,医薬品,1202200,2664.000000
9,ソースネクスト,4344,情報・通信業,879500,199.000000


In [17]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_prime, '2_東証P_DPO')

# 東証スタンダードをフィルタリングしてCSVとして保存

In [18]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [19]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[1897]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3184]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3322]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3439]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3494]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3841]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4341]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4616]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5212]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5380]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5918]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5973]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6380]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6416]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6943]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7219]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7265]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,401800,2494.0
1,住石ホールディングス,1514,鉱業,311800,350.0
2,中小企業ホールディングス,1757,建設業,954000,44.0
3,アジアゲートホールディングス,1783,建設業,634100,55.0
4,大盛工業,1844,建設業,587500,203.0
...,...,...,...,...,...
92,インプレスホールディングス,9479,情報・通信業,488800,242.0
93,アゴーラ ホスピタリティー グループ,9704,サービス業,222300,25.0
94,コックス,9876,小売業,265900,159.0
95,北沢産業,9930,卸売業,239200,388.0


In [20]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,住石ホールディングス,1514,鉱業,311800,350.000000
1,中小企業ホールディングス,1757,建設業,954000,44.000000
2,大盛工業,1844,建設業,587500,203.000000
3,ｆｏｎｆｕｎ,2323,情報・通信業,374600,519.000000
4,フォーサイド,2330,情報・通信業,410800,85.000000
5,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,370200,545.000000
6,シダー,2435,サービス業,384400,270.000000
7,フュートレック,2468,サービス業,324800,286.000000
8,ライフドリンク カンパニー,2585,食料品,333200,3445.000000
9,イメージ ワン,2667,卸売業,537600,766.000000


In [21]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,大盛工業,1844,建設業,587500,203.000000
1,ｆｏｎｆｕｎ,2323,情報・通信業,374600,519.000000
2,フォーサイド,2330,情報・通信業,410800,85.000000
3,ＹＥ ＤＩＧＩＴＡＬ,2354,情報・通信業,370200,545.000000
4,シダー,2435,サービス業,384400,270.000000
5,フュートレック,2468,サービス業,324800,286.000000
6,コメ兵ホールディングス,2780,小売業,308700,4770.000000
7,ＴＨＥグローバル社,3271,不動産業,212700,258.000000
8,クラウディアホールディングス,3607,繊維製品,995900,685.000000
9,ジェクシード,3719,情報・通信業,2889400,308.000000


In [22]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_standard, '3_東証S_UPO')

In [23]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,401800,2494.0
1,アジアゲートホールディングス,1783,建設業,634100,55.0
2,クルーズ,2138,情報・通信業,338000,913.0
3,出前館,2484,情報・通信業,447000,403.0
4,セリア,2782,小売業,206900,2296.0
5,メタプラネット,3350,卸売業,347900,22.0
6,ブロードバンドタワー,3776,情報・通信業,394200,144.0
7,サイバーステップ,3810,情報・通信業,349200,433.0
8,リミックスポイント,3825,情報・通信業,2755700,211.0
9,昭和ホールディングス,5103,ゴム製品,354100,49.0


In [24]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
0,ウエストホールディングス,1407,建設業,401800,2494.0
1,クルーズ,2138,情報・通信業,338000,913.0
2,メタプラネット,3350,卸売業,347900,22.0
3,サイバーステップ,3810,情報・通信業,349200,433.0
4,リミックスポイント,3825,情報・通信業,2755700,211.0
5,エス・サイエンス,5721,非鉄金属,224000,25.0
6,千代田化工建設,6366,建設業,1553200,354.0
7,ピクセラ,6731,電気機器,1835100,2.0
8,大黒屋ホールディングス,6993,小売業,471000,55.0
9,キムラタン,8107,繊維製品,284500,20.0


In [25]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_standard, '4_東証S_DPO')

# 東証グロースをフィルタリングしてCSVとして保存

In [26]:
# 東証スタンダード市場の銘柄コードを取得
tse_grows = get_ticker_tse('grows')
tse_grows

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,銘柄名,コード,業種
55,ルーデン・ホールディングス,1400,建設業
56,エムビーエス,1401,建設業
65,Ｌｉｂ Ｗｏｒｋ,1431,建設業
70,フィット,1436,建設業
75,ニッソウ,1444,建設業
...,...,...,...
4079,プログリット,9560,サービス業
4080,グラッドキューブ,9561,サービス業
4081,ビジネスコーチ,9562,サービス業
4082,Ａｔｌａｓ Ｔｅｃｈｎｏｌｏｇｉｅｓ,9563,サービス業


In [27]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_grows = filtered_ticker(tse_grows, min_volume=200000, max_price=5000)
filtered_tse_grows

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3261]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3913]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3986]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4445]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7362]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,304400,938.0
1,ＦＲＯＮＴＥＯ,2158,サービス業,614000,789.0
2,ジーエヌアイグループ,2160,医薬品,1196900,1400.0
3,イオレ,2334,サービス業,540500,2347.0
4,メディネット,2370,サービス業,3833000,66.0
...,...,...,...,...,...
91,ラストワンマイル,9252,サービス業,254100,4105.0
92,ＧＥＮＯＶＡ,9341,サービス業,292400,1960.0
93,ｉｓｐａｃｅ,9348,サービス業,494600,1540.0
94,リニューアブル・ジャパン,9522,電気・ガス業,2022700,840.0


In [28]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,イオレ,2334,サービス業,540500,2347.0
1,海帆,3133,小売業,8660200,811.0
2,ファンデリー,3137,小売業,225300,458.0
3,リネットジャパングループ,3556,小売業,874100,559.0
4,モブキャストホールディングス,3664,情報・通信業,1009500,80.0
5,アドバンスト・メディア,3773,情報・通信業,288300,2085.0
6,データセクション,3905,情報・通信業,559200,385.0
7,Ａｉｍｉｎｇ,3911,情報・通信業,659100,435.0
8,シェアリングテクノロジー,3989,情報・通信業,613200,727.0
9,ティアンドエス,4055,情報・通信業,200000,3025.0


In [29]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_grows = filtered_ticker(filtered_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,イオレ,2334,サービス業,540500,2347.0
1,海帆,3133,小売業,8660200,811.0
2,ファンデリー,3137,小売業,225300,458.0
3,リネットジャパングループ,3556,小売業,874100,559.0
4,アドバンスト・メディア,3773,情報・通信業,288300,2085.0
5,データセクション,3905,情報・通信業,559200,385.0
6,ティアンドエス,4055,情報・通信業,200000,3025.0
7,プレイド,4165,情報・通信業,2345600,887.0
8,サスメド,4263,情報・通信業,411800,1784.0
9,チームスピリット,4397,情報・通信業,346600,648.0


In [30]:
# CSVファイル保存
save_df_as_csv(filtered_perfect_order_week_day_tse_grows, '5_東証G_UPO')

In [31]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_grows = filtered_ticker(filtered_tse_grows, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ケアネット,2150,サービス業,304400,938.0
1,メディネット,2370,サービス業,3833000,66.0
2,メディカルネット,3645,情報・通信業,384500,410.0
3,ディー・ディー・エス,3782,情報・通信業,714000,47.0
4,ソーシャルワイヤー,3929,情報・通信業,1582600,303.0
5,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,267900,348.0
6,ココナラ,4176,情報・通信業,392900,358.0
7,アクリート,4395,情報・通信業,389900,1595.0
8,ＢＡＳＥ,4477,情報・通信業,1855300,267.0
9,オンコセラピー・サイエンス,4564,医薬品,2308600,40.0


In [32]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_grows = filtered_ticker(filtered_reverse_perfect_order_week_tse_grows,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_grows

,銘柄名,コード,業種,出来高,株価
0,ディー・ディー・エス,3782,情報・通信業,714000,47.0
1,Ｋａｉｚｅｎ Ｐｌａｔｆｏｒｍ,4170,情報・通信業,267900,348.0
2,ココナラ,4176,情報・通信業,392900,358.0
3,ＢＡＳＥ,4477,情報・通信業,1855300,267.0
4,リボミック,4591,医薬品,354900,159.0
5,ヘリオス,4593,医薬品,426600,273.0
6,窪田製薬ホールディングス,4596,医薬品,230400,103.0


In [33]:
# CSVファイル保存
save_df_as_csv(filtered_reverse_perfect_order_week_day_tse_grows, '6_東証G_DPO')

# 東証スタンダード・プライム選抜をCSVとして保存

In [46]:
import pandas as pd

def remove_duplicate_rows(dataframe, *other_dataframes):
    """重複しているティッカーコードの行を削除する

    :param dataframe: 対象のデータフレーム
    :param other_dataframes: 重複しているかどうかの判断のためのデータフレーム
    :return 引数dataframeで重複行を削除したもの
    """
    # データフレームをコピーして新しいデータフレームを作成
    result_df = dataframe.copy()

    # 重複確認のためのセットを作成
    unique_values = set()
    for df in other_dataframes:
        unique_values.update(df['コード'])

    # 重複しているティッカーコードを削除
    result_df = result_df[~result_df['コード'].isin(unique_values)]

    return result_df

In [50]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_standard_2 = filtered_ticker(tse_standard, min_volume=500000, max_price=5000)
filtered_tse_standard_2

'timestamp'
銘柄コード[1897]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3184]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3322]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3439]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3494]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3841]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4341]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4616]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5104]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5212]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5380]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5918]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5973]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[5999]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6380]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6416]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6943]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7219]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7265]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,中小企業ホールディングス,1757,建設業,954000,44.000000
1,アジアゲートホールディングス,1783,建設業,634100,55.000000
2,大盛工業,1844,建設業,587500,203.000000
3,燦キャピタルマネージメント,2134,サービス業,11488500,31.000000
4,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1531200,63.000000
5,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,568000,664.000000
6,イメージ ワン,2667,卸売業,537600,766.000000
7,レカム,3323,卸売業,645800,90.000000
8,クラウディアホールディングス,3607,繊維製品,995900,685.000000
9,ｅｎｉｓｈ,3667,情報・通信業,1606800,383.000000


In [51]:
# ティッカーコードが重複している行を削除
filtered_tse_standard_2_remove_duplicate = remove_duplicate_rows(filtered_tse_standard_2, filtered_perfect_order_week_day_tse_standard, filtered_reverse_perfect_order_week_day_tse_standard)
filtered_tse_standard_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,中小企業ホールディングス,1757,建設業,954000,44.0
1,アジアゲートホールディングス,1783,建設業,634100,55.0
3,燦キャピタルマネージメント,2134,サービス業,11488500,31.0
4,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1531200,63.0
5,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,568000,664.0
6,イメージ ワン,2667,卸売業,537600,766.0
7,レカム,3323,卸売業,645800,90.0
9,ｅｎｉｓｈ,3667,情報・通信業,1606800,383.0
10,ディー・エル・イー,3686,情報・通信業,1345100,310.0
12,インタートレード,3747,情報・通信業,782400,459.0


In [52]:
# CSVファイル保存
save_df_as_csv(filtered_tse_standard_2_remove_duplicate, '7_東証S_選抜')

In [48]:
# 東証グロースの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 50万株以上
# 株価 ... 5000円以下
filtered_tse_grows_2 = filtered_ticker(tse_grows, min_volume=500000, max_price=5000)
filtered_tse_grows_2

'timestamp'
銘柄コード[21649]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3261]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3913]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[3986]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[4445]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6067]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[6172]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[7362]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,614000,789.0
1,ジーエヌアイグループ,2160,医薬品,1196900,1400.0
2,イオレ,2334,サービス業,540500,2347.0
3,メディネット,2370,サービス業,3833000,66.0
4,ウェッジホールディングス,2388,その他金融業,8024400,117.0
5,フルッタフルッタ,2586,食料品,8288800,104.0
6,海帆,3133,小売業,8660200,811.0
7,リネットジャパングループ,3556,小売業,874100,559.0
8,モブキャストホールディングス,3664,情報・通信業,1009500,80.0
9,ＦＨＴホールディングス,3777,情報・通信業,945800,38.0


In [56]:
# ティッカーコードが重複している行を削除
filtered_tse_grows_2_remove_duplicate = remove_duplicate_rows(filtered_tse_grows_2, filtered_perfect_order_week_day_tse_grows, filtered_reverse_perfect_order_week_day_tse_grows)
filtered_tse_grows_2_remove_duplicate

,銘柄名,コード,業種,出来高,株価
0,ＦＲＯＮＴＥＯ,2158,サービス業,614000,789.0
1,ジーエヌアイグループ,2160,医薬品,1196900,1400.0
3,メディネット,2370,サービス業,3833000,66.0
4,ウェッジホールディングス,2388,その他金融業,8024400,117.0
5,フルッタフルッタ,2586,食料品,8288800,104.0
8,モブキャストホールディングス,3664,情報・通信業,1009500,80.0
9,ＦＨＴホールディングス,3777,情報・通信業,945800,38.0
12,Ａｉｍｉｎｇ,3911,情報・通信業,659100,435.0
13,ソーシャルワイヤー,3929,情報・通信業,1582600,303.0
14,シェアリングテクノロジー,3989,情報・通信業,613200,727.0


In [57]:
# CSVファイル保存
save_df_as_csv(filtered_tse_grows_2_remove_duplicate, '8_東証G_選抜')